# POWER STAGE DESIGN
By Omri Nissan <br /> Nov. 2nd, 2019

## Description
This is jupyterlab will help choose compoents appropriate for power stage of the digital converter. The requirement for the design is:

| Parameter                    | Value   |
|------                        |------   |
|   Efficiency (at Full Load)  | >90%    |
|   Output Power               | 10W     |
|   Output Voltage             | 10V     |
|   Output Current             | 1A      |
|   Input Voltage Buck         | 20V     |
|   Input Voltage Boost        | 5V      |
|   Output Voltage Ripple      | 1%      |
|   Input Voltage Ripple       | 1%      |
|   Line Regulation            | 10%     |
|   Load Regulation            | 5%      |

## Resources
Calculations of a 4 Switch Buck-Boost: http://www.ti.com/lit/an/slva535b/slva535b.pdf <br />
Calculation of a Buck Converter: http://www.ti.com/lit/an/slva477b/slva477b.pdf <br />
Calculation of a Boost Converter: http://www.ti.com/lit/an/slva372c/slva372c.pdf <br />
Gate Resistor Determination: http://www.ti.com/lit/an/slla385/slla385.pdf <br />
Current-mode Control Theory: http://www.ti.com/cn/lit/an/snva555/snva555.pdf 



In [248]:
# Import necessary libraries for the project and define output format

from astropy import units as u
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual, Layout, Box
import ipywidgets as widgets
from IPython.display import Math

from scipy import signal
import matplotlib.pyplot as plt

import pandas as pd

f = '{0:0.3f}'.format

In [255]:
# Specs
n                = 0.9          # Efficiency
L                = 220 * u.uH   # Inductor Value - NOT FINAL VALUE
V_fb             = 0.5 * u.V    # Switching Frequency  
f_sw             = 200 * u.kHz  # Switching Frequency  
P_out            = 10 * u.W     # Output Power
I_out            = 1 * u.A      # Output Current
V_out            = 10 * u.V     # Output Voltage
V_in_max         = 20 * u.V     # Maximum Input Voltage
V_in_min         = 5 * u.V      # Minmum Input Voltage
I_L_ripple       = 0.3 * I_out  # Indutor Current Ripple    
V_in_ripple_max  = 0.01         # Input Voltage Ripple
V_out_ripple_max = 0.01         # Output Voltage Ripple

In [256]:
# Feedback network based on 1% standard resistor values or less
I_fb    = 1 * u.uA
I_Rfb_2 = 100 * I_fb

R_fb2_recommended = V_fb / I_Rfb_2
print('Recommended R_fb2:', f(R_fb2_recommended.to(u.kOhm)))

R_fb1 = 1 * u.MOhm
R_fb2 = V_fb / (V_out - V_fb) * R_fb1
print('R_fb1: %s & R_fb2: %s' % (f(R_fb1.to(u.MOhm)), f(R_fb2.to(u.kOhm)))) 

R_fb2_closest = 52.3 * u.kOhm
V_out_actual = V_fb / (R_fb2_closest / (R_fb1 + R_fb2_closest))
print('V_out w/ error:', f(V_out_actual.to(u.V)))

Recommended R_fb2: 5.000 kOhm
R_fb1: 1.000 MOhm & R_fb2: 52.632 kOhm
V_out w/ error: 10.060 V


In [257]:
# Duty Cycle Definition

# --------------------------------- Buck ------------------------------------------
D_buck  = V_out / (V_in_max * n)
print('Duty cycle for Buck converter is:', f(D_buck))

# --------------------------------- Boost -----------------------------------------
D_boost = 1 - ((V_in_min * n) / V_out)
print('Duty cycle for Boost converter is:', f(D_boost))

Duty cycle for Buck converter is: 0.556
Duty cycle for Boost converter is: 0.550


In [258]:
# Inductor Choice 

# Buck Converter Calculations
L_buck = (V_out * (V_in_max - V_out)) / (I_L_ripple * f_sw * V_in_max)
print('For Buck converter the inductor must be bigger than:', f(L_buck.to(u.uH)))

# Boost Converter Calculations
L_boost = (V_in_min**2 * (V_out - V_in_min)) / (I_L_ripple * f_sw * V_out**2)
print('For Boost converter the inductor must be bigger than:', f(L_boost.to(u.uH)))

L                 = 100 * u.uH
I_buck_L_ripple   = (V_out * (V_in_max - V_out)) / (L * f_sw * V_in_max)
I_buck_L_current  = I_out + I_buck_L_ripple/2
I_boost_L_ripple  = (V_in_min**2 * (V_out - V_in_min)) / (L * f_sw * V_out**2)
I_boost_L_current = P_out/V_in_max + I_boost_L_ripple/2
print('Inductor Value choosen is:', f(L.to(u.uH)))
print('Buck conveter current ripple is %s and max current of the indcutor is %s' % (f(I_buck_L_ripple.to(u.A)), f(I_buck_L_current.to(u.A))))
print('Boost conveter current ripple is %s and max current of the indcutor is %s' % (f(I_boost_L_ripple.to(u.A)), f(I_boost_L_current.to(u.A))))

For Buck converter the inductor must be bigger than: 83.333 uH
For Boost converter the inductor must be bigger than: 20.833 uH
Inductor Value choosen is: 100.000 uH
Buck conveter current ripple is 0.250 A and max current of the indcutor is 1.125 A
Boost conveter current ripple is 0.062 A and max current of the indcutor is 0.531 A


In [253]:
# Switch Choice 
I_lim = 2 * u.A

# Buck Converter Calculations
I_buck_delta      = ((V_in_max - V_out) * D_buck) / (f_sw * L)
I_buck_sw_max     = I_out + I_buck_delta/2
I_buck_max_out    = I_lim - I_buck_delta/2
print('For Buck converter the max current is:', f(I_buck_sw_max.to(u.A)))


# Boost Converter Calculations
I_boost_delta       = (V_in_min * D_buck) / (f_sw * L)
I_boost_sw_max      = (I_out/(1-D_boost)) + I_boost_delta/2
I_boost_max_out     = (I_lim - I_boost_delta/2) * (1-D_boost)
print('For Boost converter the max current is:', f(I_boost_sw_max.to(u.A)))

print('Converter must be able to drive switch currents of at least:', f(max(I_buck_max_out,I_boost_max_out)))


For Buck converter the max current is: 1.139 A
For Boost converter the max current is: 2.292 A
Converter must be able to drive switch currents of at least: 1.861 A


In [254]:
# Capacitor Choice
'''
Notes that buck and boost have reciprocal input and output voltage characteristic, thus same output capcitance can be used to
generate the appropriate input voltage ripple.
'''

C_esr = 1 * u.mOhm

# Buck Converter Calculations
C_buck_out_1 = I_L_ripple / (8 * f_sw * V_out_ripple_max * V_out)
# C_buck_out_2 = (L * (I_L_ripple**2)) / (2 * V_out_ripple_max * V_out)
print('Buck Capcitor for 1% output voltage ripple option 1:', f(C_buck_out_1.to(u.uF)))
# print('Buck Capcitor for 1% output voltage ripple option 2:', f(C_buck_out_2.to(u.uF)))

# Boost Converter Calculations
C_boost_out_1 = (I_out * D_boost)/ (f_sw * V_out_ripple_max * V_out)
print('Boost Capcitor for 1% output voltage ripple option 1:', f(C_boost_out_1.to(u.uF)))

C_out = 50 * u.uF
C_in = 50 * u.uF

Buck Capcitor for 1% output voltage ripple option 1: 1.875 uF
Boost Capcitor for 1% output voltage ripple option 1: 27.500 uF
